In [ ]:
import sys
sys.path.append('../libs')

from esnlib import *
import pandas as pd

import tensorflow as tf

from sklearn import preprocessing
import sklearn.metrics as metrics

import sklearn.model_selection as ms
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
class tfESN():
    def __init__(self, n_reservoir = 1000, spectral_radius = 0.135, sparsity=0,
                 leaking_rate=0.3, regularization=1, random_state = None, activation = tf.tanh, initializer = tf.random_uniform):
        self.n_inputs = None
        self.n_outputs = None
        self.n_reservoir = n_reservoir
        self.spectral_radius = tf.convert_to_tensor(spectral_radius,dtype=tf.float64 )
        self.sparsity = tf.convert_to_tensor(sparsity,dtype=tf.float64)
        self.leaking_rate = tf.convert_to_tensor(leaking_rate,dtype=tf.float64)
        self.regularization = tf.convert_to_tensor(regularization,dtype=tf.float64)
        self.last_state = None
        self.activation = activation
        self.random_state = random_state
        self.initializer = initializer

    def fit(self,X,y):

        in_rows,self.n_inputs = X.get_shape().as_list()
        if len(y.get_shape().as_list()) > 1:
            out_rows, self.n_outputs = y.get_shape().as_list()
        else:
            out_rows = len(y)
            self.n_outputs = 1
        initLen = int(0.01*in_rows)

        #Raise exception
        assert(in_rows == out_rows)

        #Input length
        N = in_rows
        
        #Creating input weights
        self.Win =self.initializer([self.n_reservoir,1+self.n_inputs], seed=self.random_state, minval=-1,maxval=1, name="Win",dtype=tf.float64) 

        #Creating Reservoir weights
        self.W = self.initializer([self.n_reservoir,self.n_reservoir], seed=self.random_state, minval=-1,maxval=1, name="Win",dtype=tf.float64) 
        #Sparsity
        mask =  self.initializer([self.n_reservoir,self.n_reservoir], seed=self.random_state, minval=0,maxval=1,dtype=tf.float64) 
        self.W = tf.boolean_mask(self.W,mask > self.sparsity)
        #Spectral radius
        self.W = tf.multiply(self.W, self.spectral_radius)

        #Last state
        self.last_state  = tf.zeros(self.n_reservoir,tf.float64)

        #Collecting states
        for t in range(N):
            u = tf.gather(X,t)
            #Calculating new state
            print(tf.multiply( self.Win, tf.concat([[1],u],0)))
            self.last_state = tf.multiply(self.last_state,1-self.leaking_rate)\
                            + tf.multiply(self.leaking_rate, \
                                          self.activation( tf.multiply( self.Win, tf.concat([[1],u],0)) \
                                                                    + tf.multiply( self.W, self.last_state  ) ) )
            if t >= initLen:
                X_states[:,t-initLen] = np.hstack((1,u,self.last_state ))


#         Y_T = y[initLen:].T

#         X_sqrd = np.dot(X_states,X_states.T)+  self.regularization*np.eye(1+self.n_inputs+self.n_reservoir)
#         Y_sqrd = np.dot(Y_T,X_states.T)

#         #Getting the output weights using least squares
#         self.Wout=np.dot(Y_sqrd, np.linalg.inv(X_sqrd))

        return self

#     def predict(self,X, cont=False):
#         Y = np.empty((len(X),self.n_outputs))
#         if not cont:
#             last_state = np.zeros_like(self.last_state)
#         else:
#             last_state = self.last_state
#         for t,u in enumerate(X):
#                 last_state = (1 - self.leaking_rate) * last_state + self.leaking_rate*self.activation( np.dot( self.Win, np.hstack((1,u)))+ \
#                                                                      + np.dot( self.W, last_state  ) )
#                 y = np.dot(self.Wout, np.hstack((1,u,last_state)))
#                 Y[t,:] = y

#         if cont:
#             self.last_state = last_state
#         return Y

In [3]:
data = pd.Series.from_csv('../../data/meteo.csv')[:1000]

prediction_steps = 1
input_steps = 12
train_perc = 0.8

X,y = getDataWindowed(data,input_steps,prediction_steps)

trainlen = int(train_perc*len(X))
X_train,X_test = X[:trainlen], X[trainlen:]
y_train,y_test = y[:trainlen], y[trainlen:]

In [4]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    esn = tfESN()
    X_train = tf.convert_to_tensor(X_train,dtype=tf.float64)
    X_test = tf.convert_to_tensor(X_test,dtype=tf.float64)
    y_train = tf.convert_to_tensor(y_train,dtype=tf.float64)
    y_test = tf.convert_to_tensor(y_test,dtype=tf.float64)
    esn.fit(X_train,y_train)

Tensor("Mul_1:0", shape=(1000, 13), dtype=float64)


ValueError: Dimensions must be equal, but are 13 and 1000 for 'add' (op: 'Add') with input shapes: [1000,13], [1000].